In [35]:
import json
with open('main.json', 'r', encoding='utf8') as fp:
    data = json.load(fp)

In [36]:
import numpy as np
import core


In [37]:
metadata =[]
for k, i in data.items():
    if len(i) > 3:
        metadata.append(" ".join(i[1:3]))
    else:
        metadata.append(i[1])

In [38]:
def get_metadata(json_file, encoding='utf8'):
    """
    На вход: json-файл со словарем {индекс параграфа: [ФЗ, Глава, Статья, пункт]
    Возвращает метаданные (названия статей и глав) в формате: ['строка1', 'строка2',...]
    
    """
    with open(json_file, 'r', encoding=encoding) as fp:
        data = json.load(fp)
    metadata =[]
    for key, item in data.items():
        if len(item) > 3:
            metadata.append(" ".join(item[1:3]))
        else:
            metadata.append(item[1])
    return metadata
        

In [39]:
corp_meta = core.preprocess_corpus(metadata)

In [63]:
with open('main.json', 'r', encoding='utf8') as f:
    d = json.load(f)

In [105]:
d['1450']

['Федеральный закон № 353-ФЗ',
 'Статья 12. Уступка прав (требований) по договору потребительского кредита (займа)',
 'ч.1.']

core.top_docs_bm25okapi("обязанности оператора пдн", corp_meta, 10)


In [55]:
doc = core.load_doc('main.docx')
corp_text = core.preprocess_corpus(doc)

In [99]:
doc[1878]

'7. Полномочия органов местного самоуправления и органов государственной власти субъекта Российской Федерации в сфере рекламы, установленные настоящим Федеральным законом, могут быть перераспределены между ними в порядке, предусмотренном частью 1.2 статьи 17 Федерального закона от 6 октября 2003 года N 131-ФЗ "Об общих принципах организации местного самоуправления в Российской Федерации".'

In [10]:
import nltk
import rank_bm25
from string import punctuation
from nltk import WordPunctTokenizer
nltk.download("stopwords")
from nltk.corpus import stopwords
tokenizer = WordPunctTokenizer()
stemmer = nltk.stem.SnowballStemmer('russian')
russian_stopwords = stopwords.words("russian")

with open('syn_map.json', 'r', encoding='utf-8') as fp:
    syn_map = json.load(fp)

def preprocess_query(query, bigrams=False):
    '''

    Предобрабатываем запрос по тому же шаблону,
    что и для целого документа.
    На вход: пользовательский запрос в формате строки; на выход:
    Массив из предобработанных слов запроса
    Если bigrams=True, то функция выдаёт биграммы слов в запросе

    '''

    query = ' '.join(tokenizer.tokenize(query))
    query_text = []
    for word in query.split():
        if word not in russian_stopwords and word != ' ' and word.strip() not in punctuation and word.strip()[-1] not in punctuation:
            word = word.lower()
            stemmed_word = stemmer.stem(word)
            #если выражение из запроса в словаре syn_map, то оно заменяется на выражение из закона
            #пока может заменять только слова, а не словосочетания
            if stemmed_word in syn_map.keys():
                stemmed_word = syn_map[stemmed_word]
                if isinstance(stemmed_word, list):
                    for s_w in stemmed_word:
                        query_text.append(s_w)
            else:
                query_text.append(stemmed_word)
    if bigrams:
        query_bigrams = list(ngrams(query_text, 2))
        query_bigrams = ['_'.join(list(bigram)) for bigram in query_bigrams]
        return query_bigrams

    else:
        return query_text

def top_docs_bm25okapi_(query, corp):
    '''На вход: запрос в формате строки, предобработанный корпус текстов и количество релевантных документов в выдаче.
    На выход: индексы топ-N параграфов документа в формате массива'''
    bm25okapi = rank_bm25.BM25Okapi(corp)
    top_docs_indices = np.argsort((bm25okapi.get_scores(preprocess_query(query))))[::-1]
    top_docs_indices = list(top_docs_indices)
    return top_docs_indices


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Georgy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
def ranking_with_metadata(query, corpus_text, corpus_meta, weight_meta=0.25, ntop=5):
    
    """
    query: текстовый запрос
    corpus_text: предобработанный корпус текстов (содержания) параграфов
    corpus_meta: предобработанный корпус метаданных (названий статей и глав)
    weight_meta: веса, которые назначаются рангу документов из метаданных (названий статей и глав)
    возвращает ntop индексов параграфов, ранг которых получен путем взвешивания ранга по поиску по метаданным и ранга по поиску по содержанию
    """
    
    ranking_text = {}
    for rank, paragraph in enumerate(core.top_docs_bm25okapi(query, corpus_text, ntop=len(corpus_text))):
        ranking_text[paragraph] = rank + 1
    ranking_meta = {}
    for rank, paragraph in enumerate(core.top_docs_bm25okapi(query, corpus_meta, ntop=len(corpus_meta))):
        ranking_meta[paragraph] = rank + 1
    ranking_ = []
    for paragraph in ranking_text.keys():
        final_ranking = (weight_meta * ranking_meta[paragraph]) + ((1 - weight_meta) * ranking_text[paragraph])
        ranking_.append(tuple([final_ranking, paragraph]))
    ranking_final = [x[1] for x in sorted(ranking_, key= lambda x: x[0])]
    
    return ranking_final[:ntop]
        

In [ ]:
ranking_meta = {}
for pos, i in enumerate(top_docs_bm25okapi_('обязанности оператора персональных данных',corp_metadata)):
    ranking_meta[i] = pos + 1

In [80]:
ranking_text = {}
for pos, i in enumerate(top_docs_bm25okapi_('обязанности оператора персональных данных',corp_text)):
    ranking_text[i] = pos + 1

In [95]:
ranking_ = []
for paragraph in ranking_text.keys():
    final_ranking = (0.1 * ranking_meta[paragraph]) + (0.9 * ranking_text[paragraph])
    ranking_.append(tuple([final_ranking, paragraph]))
    

In [96]:
ranking_final = [x[1] for x in sorted(ranking_, key= lambda x: x[0])]


In [27]:
ranking_with_metadata('трансграничная передача пдн', corp_text, corp_meta)

[526, 525, 529, 527, 528]

In [28]:
core.display_passages_from_doc('трансграничная передача пдн', doc, corp_text, corp_meta, method='w')

TypeError: display_passages_from_doc() got multiple values for argument 'method'

In [29]:

import core

In [ ]:
core.display_passages_from_doc()